# Combining All Teams' Formations into a Single File

In [6]:
# Import necessary packages
import pandas as pd
from difflib import get_close_matches

# Reload all required CSVs
all_teams_df = pd.read_csv("../../data/teams/cleaned/all_league_teams.csv")

formation_files = {
    "Premier League": "../../data/teams/raw/formations/pl_team_formations.csv",
    "Serie A": "../../data/teams/raw/formations/serie_a_team_formations.csv",
    "La Liga": "../../data/teams/raw/formations/la_liga_team_formations.csv",
    "Bundesliga": "../../data/teams/raw/formations/bundesliga_team_formations.csv",
    "Ligue 1": "../../data/teams/raw/formations/ligue_1_team_formations.csv"
}

# Combine all formation data
formation_dfs = []
for league, path in formation_files.items():
    df = pd.read_csv(path)
    df["League"] = league
    formation_dfs.append(df)

formations_df = pd.concat(formation_dfs, ignore_index=True)

# Merge on both Team + League for reliability
merged_teams_df = pd.merge(all_teams_df, formations_df, left_on=["Team Name", "League"], right_on=["Team", "League"], how="left")

# Drop redundant column
merged_teams_df.drop(columns=["Team"], inplace=True)

merged_teams_df.head()

,League,Team Name,MP,GF,GA,GD,xG,xGA,xGD,Shots/90,...,Passes into Final Third/90,Passes into Penalty Area/90,Crosses into Penalty Area/90,Progressive Passes/90,Possession %,Tackles,Tackles Won,Challenges Attempted,Aerial Duel Win %,Most Common Formation
0,Premier League,Manchester City,36,67,43,24,63.6,45.3,18.3,15.89,...,17.89,4.66,0.68,20.52,61.7,475,287,430,48.4,4-2-3-1
1,Premier League,Arsenal FC,36,66,33,33,57.0,32.3,24.7,14.14,...,14.52,4.60,0.76,19.31,56.9,563,335,483,50.5,NaN
2,Premier League,Liverpool FC,36,83,37,46,77.8,34.6,43.2,16.86,...,16.54,4.57,0.88,20.03,57.6,614,380,626,52.8,NaN
3,Premier League,Chelsea FC,36,62,43,19,65.9,45.8,20.1,15.92,...,15.99,3.78,0.81,17.08,57.6,555,346,516,51.1,NaN
4,Premier League,Tottenham Hotspur,36,63,59,4,56.2,59.7,-3.5,13.53,...,13.71,4.06,0.92,17.39,55.8,639,403,569,49.8,NaN


In [7]:
# Identify teams with missing formation
teams_missing = merged_teams_df[merged_teams_df["Most Common Formation"].isna()]

# Try to fuzzy match those missing teams to formations_df["Team"]
manual_matches = {}
for team in teams_missing["Team Name"]:
    match = get_close_matches(team, formations_df["Team"].tolist(), n=1, cutoff=0.6)
    if match:
        formation = formations_df[formations_df["Team"] == match[0]]["Most Common Formation"].values[0]
        manual_matches[team] = formation

# Fill in missing formations using fuzzy matches
merged_teams_df["Most Common Formation"] = merged_teams_df.apply(
    lambda row: manual_matches.get(row["Team Name"], row["Most Common Formation"]),
    axis=1
)

# Manually filled fallback formations for the unmatched teams
manual_formations = {
    "Brighton & Hove Albion": "4-2-3-1",
    "Wolverhampton Wanderers": "3-4-3",
    "Parma Calcio 1913": "4-2-3-1",
    "RCD Espanyol Barcelona": "4-4-2",
    "Deportivo Alavés": "4-2-3-1",
    "Olympique Lyon": "4-2-3-1",
    "Stade Rennais FC": "3-4-3",
    "Stade Brestois 29": "4-3-3"
}

# Fill in missing values using the manual map
merged_teams_df["Most Common Formation"] = merged_teams_df.apply(
    lambda row: manual_formations.get(row["Team Name"], row["Most Common Formation"]),
    axis=1
)

In [8]:
output_path = "../../data/teams/cleaned/all_league_teams.csv"
merged_teams_df.to_csv(output_path, index=False)

In [9]:
formations_df.columns

Index(['Team', 'Most Common Formation', 'League'], dtype='object')